In [13]:
sys.path.insert(0, os.getcwd()+'/..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import caffe
import sys, os
from DataLoader import load_cifar

caffe.set_mode_gpu()
caffe.set_device(0)

load_cifar(datapath = '../Data')
print 'ok'

ok


In [37]:
import caffe
from caffe import layers as L
from caffe import params as P


def cnn(hdf5, batch_size):
    n = caffe.NetSpec()
    n.data, n.label_coarse, n.label_fine = L.HDF5Data(batch_size=batch_size, source=hdf5, ntop=3)
    
    n.conv1 = L.Convolution(n.data, kernel_size=4, num_output=64, weight_filler=dict(type='xavier'))
    n.cccp1a = L.Convolution(n.conv1, kernel_size=1, num_output=42, weight_filler=dict(type='xavier'))
    n.relu1a = L.ReLU(n.cccp1a, in_place=True)
    n.cccp1b = L.Convolution(n.relu1a, kernel_size=1, num_output=32, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.cccp1b, kernel_size=3, stride=2, pool=P.Pooling.MAX)
    n.drop1 = L.Dropout(n.pool1, in_place=True)
    n.relu1b = L.ReLU(n.drop1, in_place=True)
    
    n.conv2 = L.Convolution(n.relu1b, kernel_size=4, num_output=42, weight_filler=dict(type='xavier'))
    n.pool2 = L.Pooling(n.conv2, kernel_size=3, stride=2, pool=P.Pooling.MAX)
    n.drop2 = L.Dropout(n.pool2, in_place=True)
    n.relu2 = L.ReLU(n.drop2, in_place=True)
    
    n.conv3 = L.Convolution(n.relu2, kernel_size=2, num_output=64, weight_filler=dict(type='xavier'))
    n.pool3 = L.Pooling(n.conv3, kernel_size=2, stride=2, pool=P.Pooling.AVE)
    n.relu3 = L.ReLU(n.pool3, in_place=True)
    
    n.ip1 = L.InnerProduct(n.relu3, num_output=768, weight_filler=dict(type='xavier'))
    n.sig1 = L.Sigmoid(n.ip1, in_place=True)
    
    n.ip_c = L.InnerProduct(n.sig1, num_output=20, weight_filler=dict(type='xavier'))
    n.accuracy_c = L.Accuracy(n.ip_c, n.label_coarse)
    n.loss_c = L.SoftmaxWithLoss(n.ip_c, n.label_coarse)
    
    n.ip_f = L.InnerProduct(n.sig1, num_output=100, weight_filler=dict(type='xavier'))
    n.accuracy_f = L.Accuracy(n.ip_f, n.label_fine)
    n.loss_f = L.SoftmaxWithLoss(n.ip_f, n.label_fine)
    
    return n.to_proto()
    

def lenet(hdf5, batch_size):
    n = caffe.NetSpec()
    n.data, n.label_coarse, n.label_fine = L.HDF5Data(batch_size=batch_size, source=hdf5, ntop=3)
    
    n.conv1 = L.Convolution(n.data, kernel_size=3, num_output=32, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.conv2 = L.Convolution(n.pool1, kernel_size=4, num_output=500, weight_filler=dict(type='xavier'))
    n.pool2 = L.Pooling(n.conv2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.ip1 = L.InnerProduct(n.pool2, num_output=500, weight_filler=dict(type='xavier'))
    n.relu1 = L.ReLU(n.ip1, in_place=True)
    
    n.ip2 = L.InnerProduct(n.relu1, num_output=100, weight_filler=dict(type='xavier'))
    n.accuracy_f = L.Accuracy(n.ip2, n.label_fine)
    n.loss = L.SoftmaxWithLoss(n.ip2, n.label_fine)
    return n.to_proto()

    
with open('cnn_train.prototxt', 'w') as f:
    f.write(str(lenet('train.txt', 100)))
    
with open('cnn_test.prototxt', 'w') as f:
    f.write(str(lenet('test.txt', 120)))

solver = caffe.get_solver('cnn_solver_rms.prototxt')

print("Layers' features:")
[(k, v.data.shape) for k, v in solver.net.blobs.items()]

Layers' features:


[('data', (100, 3, 32, 32)),
 ('label_coarse', (100,)),
 ('label_fine', (100,)),
 ('label_fine_data_2_split_0', (100,)),
 ('label_fine_data_2_split_1', (100,)),
 ('conv1', (100, 20, 29, 29)),
 ('pool1', (100, 20, 14, 14)),
 ('conv2', (100, 500, 11, 11)),
 ('pool2', (100, 500, 6, 6)),
 ('ip1', (100, 500)),
 ('ip2', (100, 100)),
 ('ip2_ip2_0_split_0', (100, 100)),
 ('ip2_ip2_0_split_1', (100, 100)),
 ('accuracy_f', ()),
 ('loss', ())]

In [31]:
print("Parameters and shape:")
[(k, v[0].data.shape) for k, v in solver.net.params.items()]

Parameters and shape:


[('conv1', (20, 3, 5, 5)),
 ('conv2', (50, 20, 5, 5)),
 ('ip1', (500, 1250)),
 ('ip2', (100, 500))]

In [32]:
!cat cnn_solver_rms.prototxt

train_net: "cnn_train.prototxt"
test_net: "cnn_test.prototxt"

test_iter: 100
test_interval: 1000

base_lr: 0.0006
momentum: 0.0
weight_decay: 0.001

lr_policy: "inv"
gamma: 0.0001
power: 0.75

display: 100

max_iter: 1000

snapshot: 50000
snapshot_prefix: "cnn_snapshot"
solver_mode: GPU

type: "RMSProp"
rms_decay: 0.98


In [38]:
%%time
!$CAFFE_ROOT/build/tools/caffe train -solver cnn_solver_rms.prototxt

I0210 18:24:27.752063 25601 caffe.cpp:185] Using GPUs 0
I0210 18:24:27.767523 25601 caffe.cpp:190] GPU 0: GeForce GTX 860M
I0210 18:24:27.925062 25601 solver.cpp:48] Initializing solver from parameters: 
train_net: "cnn_train.prototxt"
test_net: "cnn_test.prototxt"
test_iter: 100
test_interval: 1000
base_lr: 0.0006
display: 100
max_iter: 1000
lr_policy: "inv"
gamma: 0.0001
power: 0.75
momentum: 0
weight_decay: 0.001
snapshot: 50000
snapshot_prefix: "cnn_snapshot"
solver_mode: GPU
device_id: 0
rms_decay: 0.98
type: "RMSProp"
I0210 18:24:27.925300 25601 solver.cpp:81] Creating training net from train_net file: cnn_train.prototxt
I0210 18:24:27.925725 25601 net.cpp:49] Initializing net from parameters: 
state {
  phase: TRAIN
}
layer {
  name: "data"
  type: "HDF5Data"
  top: "data"
  top: "label_coarse"
  top: "label_fine"
  hdf5_data_param {
    source: "train.txt"
    batch_size: 100
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param

In [39]:
%%time
!$CAFFE_ROOT/build/tools/caffe test -model cnn_test.prototxt -weights cnn_snapshot_iter_1500.caffemodel -iterations 100

I0210 18:29:33.021010 25775 caffe.cpp:246] Use CPU.
I0210 18:29:33.194944 25775 net.cpp:49] Initializing net from parameters: 
state {
  phase: TEST
}
layer {
  name: "data"
  type: "HDF5Data"
  top: "data"
  top: "label_coarse"
  top: "label_fine"
  hdf5_data_param {
    source: "test.txt"
    batch_size: 120
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 20
    kernel_size: 4
    weight_filler {
      type: "xavier"
    }
  }
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "conv2"
  type: "Convolution"
  bottom: "pool1"
  top: "conv2"
  convolution_param {
    num_output: 500
    kernel_size: 4
    weight_filler {
      type: "xavier"
    }
  }
}
layer {
  name: "pool2"
  type: "Pooling"
  bottom: "conv2"
  top: "pool2"
  pooling_param {
    pool: MAX
    kernel_size: 2
    stride: 2
  }
}
layer 